In [1]:
import pandas as pd
import glob


import os
from tqdm import tqdm
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3

## Import all CSVs from 2020 - 2014 into a single DF.

In [ ]:
%%time

path = r'./data_airQA' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True, sort=False)

### Save to CSV

In [ ]:
# frame.to_csv('data_airQA/aws_raw.csv') 

## Filter the raw data for Seattle and re-export as a CSV

In [ ]:
%%time

df = pd.read_csv("data_airQA/aws_raw.csv")

In [ ]:
df.loc[df['city']=='Seattle-Tacoma-Bellevue']

In [ ]:
%%time

# Reference: https://pythondata.com/working-large-csv-files-python/

    
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! # ONE TIME process only! # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! #
                # Export CSV to a SQLITE DB, so that queries are manageable in-memory.
    

chunksize = 10 ** 6 # Chuncking the csv by 1 million rows per iterations

file = 'data_airQA/aws_raw.csv'

csv_database = create_engine('sqlite:///data_airQA.db') # Generated a 3.0GB file!

i = 0
j = 1
for df in tqdm(pd.read_csv(file, chunksize=chunksize, iterator=True)):
      df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
      df.index += j
      i+=1
      df.to_sql('table', csv_database, if_exists='append')
      j = df.index[-1] + 1

### Filter the dataset to retain data solely for the Seattle area

In [ ]:
df = df.loc[df['city']=='Seattle-Tacoma-Bellevue']

In [ ]:
# df.to_csv('data_airQA/aws_Seattle.csv') 

## Wrangling the dataset

In [ ]:
df_raw = pd.read_csv("data_airQA/aws_Seattle.csv")

In [ ]:
print(df_raw.shape)
print('-----------')
print(df_raw.dtypes)

df_raw.head()

### Retain the parameter for PM2.5

In [ ]:
df = df_raw.loc[df_raw['parameter']=='pm25']
df = df.dropna(subset=['value'], how='all') # there were none


print(df.shape)
print('-----------')
print(str('We are left with ') + str(float((df.shape[0] / df_raw.shape[0])) * 100) + str('% of the raw dataset after parsing for PM2.5'))
df.head()

### Extract Date (local time, not UTC) & set field to DateTime

In [ ]:
df['year'] = df['date'].str.split('=').str[1].str.split('-').str[0]

df['date_drop'] =  df.date.str.split(', local=').str[1]
df['date'] = df['date_drop'].str.split('T').str[0]
df['hour'] = df['date_drop'].str.split('T').str[1]
df['hour'] = df['hour'].str.replace(r':00-08:00}', '')

df['dateTime'] = pd.to_datetime(df['date'] + ' ' + df['hour'])

In [ ]:
### Split Latitude and Longitude into separate fields

df['Latitude'] = df.coordinates.str.split(', longitude=').str[0].str.replace(r'{latitude=', '')
df['Longitude'] = df.coordinates.str.split(', longitude=').str[1].str.replace(r'}', '')

In [ ]:
# Reformat the attribution field 

df['attribution'] = 'US EPA AirNow, http://www.airnow.gov/, Washington Department of Ecology'

In [ ]:
df = df.drop(['country','sourcename','sourcetype','mobile','coordinates',
              'averagingperiod','Unnamed: 0','Unnamed:0','date_drop'],axis=1)

In [ ]:
print('Cleaned Master Dataset')
print('----------')
print(df.shape)
print('----------')
print(df.dtypes)

df.head()

### Save cleaned 'MasterDataset' to CSV

In [ ]:
# df.to_csv('data_airQA/aws_Seattle_clean.csv')

### Set the Date and Time as the dataframe's index, and sort the dataframe chronologically

In [ ]:
df = df.sort_values(by='dateTime')
df.set_index(['dateTime'], inplace = True)

In [ ]:
print('The dataframe ranges from 2016-03-14 21:00:00 to 2020-05-29 17:00:00')

df.head()

## Timeseries

In [ ]:
df_plot = df.groupby(['Latitude','Longitude','location','date','year','dateTime']).mean().reset_index()

df_20 = df_plot[df_plot['year']=='2020']
df_20.set_index(['dateTime'], inplace = True)
df_20 = df_20.resample('D').mean()
# df_20['time'] = df_20.index.strftime('%m-%d')
# df_20 = df_20.reset_index()

df_19 = df_plot[df_plot['year']=='2019']
df_19.set_index(['dateTime'], inplace = True)
df_19 = df_19.resample('D').mean()
# df_19['time'] = df_19.index.strftime('%m-%d')
# df_19 = df_19.reset_index()


df_18 = df_plot[df_plot['year']=='2018']
df_18.set_index(['dateTime'], inplace = True)
df_18 = df_18.resample('D').mean()
# df_18['time'] = df_18.index.strftime('%m-%d')
# df_18 = df_18.reset_index()

df_17 = df_plot[df_plot['year']=='2017']
df_17.set_index(['dateTime'], inplace = True)
df_17 = df_17.resample('D').mean()
# df_17['time'] = df_17.index.strftime('%m-%d')
# df_17 = df_17.reset_index()

df_16 = df_plot[df_plot['year']=='2016']
df_16.set_index(['dateTime'], inplace = True)
df_16 = df_16.resample('D').mean()
# df_16['time'] = df_16.index.strftime('%m-%d')
# df_16 = df_16.reset_index()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import *
from matplotlib import *

plt.figure(figsize=(20,13))
with plt.xkcd():
    sns.set_style('darkgrid')
    sns.lineplot(x=df_20.index, y= df_20.value, marker='o',alpha=1, color='blue')
    sns.lineplot(x=df_19.index, y= df_19.value, marker='o',alpha=1, color='green')
    sns.lineplot(x=df_18.index, y= df_18.value, marker='o',alpha=1, color='red')
    sns.lineplot(x=df_17.index, y= df_17.value, marker='o',alpha=1, color='orange')
    sns.lineplot(x=df_16.index, y= df_16.value, marker='o',alpha=1, color='purple')

    ax = pylab.axes()
    ax.set_title("PM2.5 Measurements, Seattle Metro Area, Jan. - May")

    ax.set_ylabel('µg/m³')
    ax.set_xlabel('Time of Measurement')